# Exercise 1: Infer global numerical diffusivity from globally integrated variance budget
The globally integrated variance budget is given by:
$$ \frac{\partial}{\partial t} \iint \frac{q^2}{2} dA = -\kappa_{num} \iint |\nabla q^2| \,dA$$

We can estimate the tendency term on the LHS using the snapshot diagnostics and the gradient variance term on thr RHS using the mean diagnostics, and solve for $\kappa_{num}$. This is only possible under the assumption that $\kappa_{num}$ is spatially constant.

In [1]:
import xarray as xr
import numpy as np
from xmitgcm import open_mdsdataset
from xgcm import Grid
from os.path import join as pjoin
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from dask.distributed import Client
client=Client(scheduler_file='/rigel/home/jb3210/scheduler.json')
client

Client Scheduler: tcp://10.43.4.194:8786 Dashboard: http://10.43.4.194:8787,Cluster Workers: 41 Cores: 164 Memory: 492.00 GB


In [3]:
def infer_kappa_from_global_variance_budget(snap, mean, grid):
    dt = snap['TRACSQ'].time[0:2].diff('time').data
    LHS = ((snap['TRACSQ']/2).diff('time')/dt*snap['rA']).sum(['i', 'j'])
    RHS = -((grid.interp(mean['DXSqTr'], 'X') + \
            grid.interp(mean['DYSqTr'], 'Y'))*ds_mean['rA']).sum(['i', 'j'])
    return LHS/RHS

In [4]:
def convert_trnum2dimension(ds):
    def drop_nonmatching_vars(ds, var_list):
        data_vars = list(ds.data_vars)
        drop_vars = [a for a in data_vars if a not in var_list]
        return ds.drop(drop_vars)
    
    def rename_vars(ds):
        for vv in list(ds.data_vars):
            ds = ds.rename({vv:vv[0:-2]})
        return ds
        
    tr_num = list(set([a[-2:] for a in list(ds.data_vars)]))
    tr_num.sort()
    tr_vars = list(set([a[:-2] for a in list(ds.data_vars)]))
    
    datasets = [drop_nonmatching_vars(ds, [a+n for a in tr_vars]) for n in tr_num]
    datasets = [rename_vars(b) for b in datasets]
    
    tr_num_int = [int(a) for a in tr_num]
    
    tr_dim = xr.DataArray(tr_num_int, 
                          dims='tracer_no',
                          coords={'tracer_no': (['tracer_no', ], tr_num_int)})
    
    ds_combined = xr.concat(datasets,dim='tracer_no')
    ds_combined['tracer_no'] = tr_dim
    
    return ds_combined

In [6]:
ddir = '/rigel/ocp/users/jb3210/projects/aviso_surface_tracer/runs'
timestep = 900 # in seconds
chunksize = 1000
readin_dict = dict(delta_t=timestep, swap_dims=False, chunks={'i':chunksize,'j':chunksize,
                                                              'i_g':chunksize,'j_g':chunksize})
kappa = 63

# for rr in ['PSI', 'LAT', 'SSS', 'SST']:
for rr in ['LAT']:
    run = 'run_KOC_%s_variance_budget' %rr
    rundir = pjoin(ddir,run)
    
    ds_mean = convert_trnum2dimension(open_mdsdataset(rundir,prefix=['tracer_diags'],
                                                  **readin_dict))

    ds_snap = convert_trnum2dimension(open_mdsdataset(rundir,prefix=['tracer_snapshots'],
                                                      **readin_dict))
    grid = Grid(ds_mean)
    
    # Determine the numerical diffusivity from globally integrated variance budget
    inferred_kappa = infer_kappa_from_global_variance_budget(ds_snap, ds_mean, grid).compute()
    plt.figure(figsize=[8,2])
    for tt in inferred_kappa.tracer_no:
        inferred_kappa.sel(tracer_no=tt).plot()
    plt.gca().set_ylim([0,10])
    plt.axhline(0, color='0.7')
    plt.ylabel('kappa [m/s^2]')
    plt.title('%s numerical diffusivity inferred from global variance budget' %rr)
    plt.legend(['tracer%02i' %a for a in ds_mean.tracer_no.data])

/rigel/home/jb3210/src/xmitgcm/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/rigel/home/jb3210/src/xmitgcm/xmitgcm/mds_store.py:220: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
distributed.utils - ERROR - ('truediv-a6c1808269664f0b185f550a6a20239c', 1, 45)
Traceback (most recent call last):
  File "/rigel/home/jb3210/code/miniconda/envs/standard/lib/python3.6/site-packages/distributed/client.py", line 1305, in _gather
    st = self.futures[key]
KeyError: "('truediv-a6c1808269664f0b185f550a6a20239c', 1, 45)"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/rigel/home/jb3210/code/miniconda/envs/standard/lib/python3.6/site-packages/

KeyboardInterrupt: 